In [111]:
import os
import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from func import *
import json
import ast

### 1. annotation 가져오기

In [112]:
# JSON 파일 경로
json_file_path = 'person_keypoints_val2017.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    data = json.load(f)

In [113]:
data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [114]:
data['annotations'][0].keys()

dict_keys(['segmentation', 'num_keypoints', 'area', 'iscrowd', 'keypoints', 'image_id', 'bbox', 'category_id', 'id'])

In [115]:
image_id = []
bbox = []
id = []

for item in data['annotations']:
    image_id.append(item['image_id'])
    bbox.append(item['bbox'])
    id.append(item['id'])

In [116]:
gt_df = pd.DataFrame({'image_id':image_id, 'bbox':bbox})
gt_df

,image_id,bbox
0,425226,"[73.35, 206.02, 300.58, 372.5]"
1,440475,"[491.91, 183.51, 34.85, 73.4]"
2,266400,"[36.32, 0.59, 38.56, 53.86]"
3,69213,"[276.35, 9.6, 244.69, 400.13]"
4,555050,"[97.1, 185.73, 28.66, 27.72]"
...,...,...
10999,259690,"[27, 98, 225, 114]"
11000,145020,"[27, 201, 209, 68]"
11001,562818,"[353, 229, 180, 39]"
11002,439994,"[0, 0, 427, 458]"


In [117]:
gt_df["width"] = gt_df["bbox"].apply(lambda bbox: bbox[2])
gt_df["height"] = gt_df["bbox"].apply(lambda bbox: bbox[3])
gt_df["bbox_size"] = gt_df["width"]*gt_df["height"]

In [118]:
gt_df

,image_id,bbox,width,height,bbox_size
0,425226,"[73.35, 206.02, 300.58, 372.5]",300.58,372.50,111966.0500
1,440475,"[491.91, 183.51, 34.85, 73.4]",34.85,73.40,2557.9900
2,266400,"[36.32, 0.59, 38.56, 53.86]",38.56,53.86,2076.8416
3,69213,"[276.35, 9.6, 244.69, 400.13]",244.69,400.13,97907.8097
4,555050,"[97.1, 185.73, 28.66, 27.72]",28.66,27.72,794.4552
...,...,...,...,...,...
10999,259690,"[27, 98, 225, 114]",225.00,114.00,25650.0000
11000,145020,"[27, 201, 209, 68]",209.00,68.00,14212.0000
11001,562818,"[353, 229, 180, 39]",180.00,39.00,7020.0000
11002,439994,"[0, 0, 427, 458]",427.00,458.00,195566.0000


### 2. model result에서 score 가져오기

In [119]:
# JSON 파일 경로
json_file_path = 'result_keypoints.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    result = json.load(f)

In [120]:
result[0].keys()

dict_keys(['category_id', 'center', 'image_id', 'keypoints', 'scale', 'score'])

In [121]:
len(result)

6334

In [122]:
center = []
image_id = []
scale = []
score = []

for item in result:
    center.append(item['center'])
    image_id.append(item['image_id'])
    scale.append(item['scale'])
    score.append(item['score'])

In [123]:
result_df = pd.DataFrame({'image_id':image_id,'center':center,'scale':scale,'score':score})
result_df

,image_id,center,scale,score
0,397133,"[442.864990234375, 208.22999572753906]","[1.2966562509536743, 1.7288750410079956]",0.856345
1,397133,"[30.579999923706055, 280.69500732421875]","[0.38224998116493225, 0.5096666812896729]",0.397612
2,252219,"[361.3999938964844, 272.68499755859375]","[0.919921875, 1.2265625]",0.895720
3,252219,"[70.26000213623047, 279.7850036621094]","[1.056796908378601, 1.409062385559082]",0.895242
4,252219,"[571.77001953125, 278.6499938964844]","[1.0066875219345093, 1.3422499895095825]",0.829179
...,...,...,...,...
6329,15335,"[286.7699890136719, 99.50499725341797]","[0.6165000200271606, 0.8219999670982361]",0.508098
6330,15335,"[533.0349731445312, 99.5199966430664]","[0.601312518119812, 0.8017499446868896]",0.415471
6331,15335,"[426.94000244140625, 109.0199966430664]","[1.0413750410079956, 1.3884999752044678]",0.396422
6332,15335,"[198.88499450683594, 108.5250015258789]","[0.48118752241134644, 0.6415833234786987]",0.294983


- image_id -> score 정렬되어있음

In [124]:
# # 데이터를 평평한 형태로 변환
# flat_data = [(image_id, bbox_data) for image_id, bbox_data_list in result_df.items() for bbox_data in bbox_data_list]

# # 데이터프레임으로 변환
# result_df = pd.DataFrame(flat_data, columns=['image_id', 'bbox_data'])

# # JSON normalization을 통해 nested된 데이터를 펼침
# df_normalized = pd.json_normalize(result_df['bbox_data'])

# # 기존 'bbox_data' 열 제거 및 새로운 열로 추가
# result_df = pd.concat([result_df.drop(['bbox_data'], axis=1), df_normalized], axis=1)

# # 결과 확인
# print(result_df)

In [125]:
result_df.columns

Index(['image_id', 'center', 'scale', 'score'], dtype='object')

In [126]:
# tmp = result_df[['image_id', 'center', 'scale', 'score']]
# result_df = tmp.iloc[:,1:]
# result_df.columns

In [127]:
# bbox에서 kpts 저장되어있는 모델 중간 json 파일
# JSON 파일 경로
json_file_path = 'GT_result.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    kpt_result = json.load(f)

In [128]:
kpt_result['397133'][0].keys()

dict_keys(['keypoints', 'center', 'scale', 'area', 'score', 'image_id', 'bbox_id'])

In [129]:
# 데이터프레임을 담을 리스트 생성
df_list = []

# JSON 데이터를 데이터프레임으로 변환
for image_id, bbox_list in kpt_result.items():
    for bbox_id, bbox_info in enumerate(bbox_list):
        # 필요한 정보 추출
        bbox_data = {
            "image_id": int(image_id),
            "bbox_id": bbox_id,
            "keypoints": bbox_info["keypoints"]["__ndarray__"],
            "center": bbox_info["center"]["__ndarray__"],
            "scale": bbox_info["scale"]["__ndarray__"],
            "score": bbox_info["score"],
            "area" :bbox_info["area"]
        }
        df_list.append(bbox_data)

# 리스트를 데이터프레임으로 변환
kpt_df = pd.DataFrame(df_list)

In [130]:
print(kpt_df.columns)
print(len(kpt_df))

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'score', 'area'], dtype='object')
6352


In [131]:
print(kpt_df.columns)
print(result_df.columns)
print(gt_df.columns)

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'score', 'area'], dtype='object')
Index(['image_id', 'center', 'scale', 'score'], dtype='object')
Index(['image_id', 'bbox', 'width', 'height', 'bbox_size'], dtype='object')


In [132]:
result_df['center']

0         [442.864990234375, 208.22999572753906]
1       [30.579999923706055, 280.69500732421875]
2        [361.3999938964844, 272.68499755859375]
3         [70.26000213623047, 279.7850036621094]
4           [571.77001953125, 278.6499938964844]
                          ...                   
6329      [286.7699890136719, 99.50499725341797]
6330       [533.0349731445312, 99.5199966430664]
6331     [426.94000244140625, 109.0199966430664]
6332     [198.88499450683594, 108.5250015258789]
6333      [578.4849853515625, 91.56999969482422]
Name: center, Length: 6334, dtype: object

In [133]:
kpt_df['center']

0         [442.864990234375, 208.22999572753906]
1       [30.579999923706055, 280.69500732421875]
2        [361.3999938964844, 272.68499755859375]
3         [70.26000213623047, 279.7850036621094]
4           [571.77001953125, 278.6499938964844]
                          ...                   
6347                 [107.06500244140625, 259.5]
6348       [533.0349731445312, 99.5199966430664]
6349      [578.4849853515625, 91.56999969482422]
6350     [426.94000244140625, 109.0199966430664]
6351     [198.88499450683594, 108.5250015258789]
Name: center, Length: 6352, dtype: object

In [134]:
# 리스트를 풀어서 각각의 요소를 새로운 열로 만들기
kpt_df_expanded = kpt_df['center'].apply(pd.Series).rename(columns={0:'center_x',1:'center_y'})
result_df_expanded = result_df['center'].apply(pd.Series).rename(columns={0:'center_x',1:'center_y'})

In [135]:
# 기존의 key 열은 삭제
df1 = pd.concat([kpt_df, kpt_df_expanded], axis=1)
df2 = pd.concat([result_df, result_df_expanded], axis=1)

# 키를 기준으로 데이터프레임 합치기
merged_df = pd.merge(df1, df2, how='inner', on=['center_x', 'center_y'], suffixes=('_df1', '_df2'))

In [136]:
merged_df.columns

Index(['image_id_df1', 'bbox_id', 'keypoints', 'center_df1', 'scale_df1',
       'score_df1', 'area', 'center_x', 'center_y', 'image_id_df2',
       'center_df2', 'scale_df2', 'score_df2'],
      dtype='object')

In [137]:
merged_df[merged_df['image_id_df1'] != merged_df['image_id_df2']]

,image_id_df1,bbox_id,keypoints,center_df1,scale_df1,score_df1,area,center_x,center_y,image_id_df2,center_df2,scale_df2,score_df2


In [141]:
# 중복열 제거
merged_df.drop(columns=['image_id_df2', 'scale_df2', 'center_df2', 'score_df1', 'center_x','center_y'], inplace=True, axis=1)

In [142]:
merged_df.rename(columns={'image_id_df1':'image_id', 'center_df1':'center', 'scale_df1':'scale', 'score_df2':'score'}, inplace=True)
print(merged_df.shape)
print(merged_df.head())

(6334, 7)
   image_id  bbox_id                                          keypoints  \
0    397133        0  [[430.8645324707031, 95.42893981933594, 0.9389...   
1    397133        1  [[52.14830780029297, 284.97412109375, 0.023037...   
2    252219        0  [[354.296630859375, 198.06585693359375, 0.9260...   
3    252219        1  [[100.3197021484375, 189.2662811279297, 0.9693...   
4    252219        2  [[537.075927734375, 193.65164184570312, 0.9272...   

                                     center  \
0    [442.864990234375, 208.22999572753906]   
1  [30.579999923706055, 280.69500732421875]   
2   [361.3999938964844, 272.68499755859375]   
3    [70.26000213623047, 279.7850036621094]   
4      [571.77001953125, 278.6499938964844]   

                                       scale          area     score  
0   [1.2966562509536743, 1.7288750410079956]  89670.257812  0.856345  
1  [0.38224998116493225, 0.5096666812896729]   7792.803223  0.397612  
2                   [0.919921875, 1.2265625

In [143]:
gt_df.columns

Index(['image_id', 'bbox', 'width', 'height', 'bbox_size'], dtype='object')

In [144]:
gt_df

,image_id,bbox,width,height,bbox_size
0,425226,"[73.35, 206.02, 300.58, 372.5]",300.58,372.50,111966.0500
1,440475,"[491.91, 183.51, 34.85, 73.4]",34.85,73.40,2557.9900
2,266400,"[36.32, 0.59, 38.56, 53.86]",38.56,53.86,2076.8416
3,69213,"[276.35, 9.6, 244.69, 400.13]",244.69,400.13,97907.8097
4,555050,"[97.1, 185.73, 28.66, 27.72]",28.66,27.72,794.4552
...,...,...,...,...,...
10999,259690,"[27, 98, 225, 114]",225.00,114.00,25650.0000
11000,145020,"[27, 201, 209, 68]",209.00,68.00,14212.0000
11001,562818,"[353, 229, 180, 39]",180.00,39.00,7020.0000
11002,439994,"[0, 0, 427, 458]",427.00,458.00,195566.0000


In [145]:
# 기존의 _xywh2cs 함수 정의
def _xywh2cs(x, y, w, h, padding=1.25):
    aspect_ratio = 1.0  # 임의로 aspect_ratio를 1.0으로 설정
    center = np.array([x + w * 0.5, y + h * 0.5], dtype=np.float32)

    if np.random.rand() < 0.3:
        center += 0.4 * (np.random.rand(2) - 0.5) * [w, h]

    if w > aspect_ratio * h:
        h = w * 1.0 / aspect_ratio
    elif w < aspect_ratio * h:
        w = h * aspect_ratio

    scale = np.array([w / 200.0, h / 200.0], dtype=np.float32)
    scale = scale * padding

    return center, scale

In [146]:
# bbox 열을 기반으로 x, y, width, height 열을 생성
gt_df[['x', 'y', 'width', 'height']] = pd.DataFrame(gt_df['bbox'].tolist(), index=gt_df.index)

# _xywh2cs 함수를 이용하여 center, scale 계산
gt_df[['center', 'scale']] = gt_df.apply(lambda row: pd.Series(_xywh2cs(row['x'], row['y'], row['width'], row['height'])), axis=1)


print(gt_df)

       image_id                            bbox   width  height    bbox_size  \
0        425226  [73.35, 206.02, 300.58, 372.5]  300.58  372.50  111966.0500   
1        440475   [491.91, 183.51, 34.85, 73.4]   34.85   73.40    2557.9900   
2        266400     [36.32, 0.59, 38.56, 53.86]   38.56   53.86    2076.8416   
3         69213   [276.35, 9.6, 244.69, 400.13]  244.69  400.13   97907.8097   
4        555050    [97.1, 185.73, 28.66, 27.72]   28.66   27.72     794.4552   
...         ...                             ...     ...     ...          ...   
10999    259690              [27, 98, 225, 114]  225.00  114.00   25650.0000   
11000    145020              [27, 201, 209, 68]  209.00   68.00   14212.0000   
11001    562818             [353, 229, 180, 39]  180.00   39.00    7020.0000   
11002    439994                [0, 0, 427, 458]  427.00  458.00  195566.0000   
11003    250282               [0, 34, 639, 388]  639.00  388.00  247932.0000   

            x       y                  

In [147]:
merged_df

,image_id,bbox_id,keypoints,center,scale,area,score
0,397133,0,"[[430.8645324707031, 95.42893981933594, 0.9389...","[442.864990234375, 208.22999572753906]","[1.2966562509536743, 1.7288750410079956]",89670.257812,0.856345
1,397133,1,"[[52.14830780029297, 284.97412109375, 0.023037...","[30.579999923706055, 280.69500732421875]","[0.38224998116493225, 0.5096666812896729]",7792.803223,0.397612
2,252219,0,"[[354.296630859375, 198.06585693359375, 0.9260...","[361.3999938964844, 272.68499755859375]","[0.919921875, 1.2265625]",45133.667969,0.895720
3,252219,1,"[[100.3197021484375, 189.2662811279297, 0.9693...","[70.26000213623047, 279.7850036621094]","[1.056796908378601, 1.409062385559082]",59563.707031,0.895242
4,252219,2,"[[537.075927734375, 193.65164184570312, 0.9272...","[571.77001953125, 278.6499938964844]","[1.0066875219345093, 1.3422499895095825]",54049.058594,0.829179
...,...,...,...,...,...,...,...
6329,15335,3,"[[83.04756164550781, 175.28688049316406, 0.925...","[107.06500244140625, 259.5]","[1.7588437795639038, 2.3451249599456787]",164988.343750,0.714482
6330,15335,4,"[[508.6524963378906, 30.947364807128906, 0.216...","[533.0349731445312, 99.5199966430664]","[0.601312518119812, 0.8017499446868896]",19284.091797,0.415471
6331,15335,5,"[[561.7669067382812, 92.91932678222656, 0.1978...","[578.4849853515625, 91.56999969482422]","[0.6596249938011169, 0.8794999718666077]",23205.607422,0.282995
6332,15335,6,"[[408.19903564453125, 93.58609008789062, 0.499...","[426.94000244140625, 109.0199966430664]","[1.0413750410079956, 1.3884999752044678]",57837.964844,0.396422


In [148]:
gt_df = gt_df.sort_values(by='image_id', ascending=True).reset_index(drop=True)

print(gt_df)

       image_id                              bbox   width  height  \
0           139    [384.43, 172.21, 15.12, 35.74]   15.12   35.74   
1           139    [412.8, 157.61, 53.05, 138.01]   53.05  138.01   
2           785    [280.79, 44.73, 218.7, 346.68]  218.70  346.68   
3           872  [145.26, 100.67, 291.95, 457.35]  291.95  457.35   
4           872   [163.73, 126.42, 265.69, 480.4]  265.69  480.40   
...         ...                               ...     ...     ...   
10999    581357    [578.74, 389.17, 12.86, 25.39]   12.86   25.39   
11000    581357   [262.68, 93.29, 220.04, 247.55]  220.04  247.55   
11001    581357     [249.22, 419.65, 13.2, 24.53]   13.20   24.53   
11002    581357    [428.92, 400.24, 22.69, 45.38]   22.69   45.38   
11003    581357        [523.6, 404.09, 12.73, 37]   12.73   37.00   

         bbox_size       x       y                  center  \
0         540.3888  384.43  172.21  [393.34488, 193.39546]   
1        7321.4305  412.80  157.61      [439.32

In [149]:
# 두 개의 열을 기준으로 정렬 (예: Column1을 오름차순으로 정렬하고, 그 후에 Column2를 오름차순으로 정렬)
merged_df = merged_df.sort_values(by=['image_id', 'score'], ascending=[True, True]).reset_index(drop=True)

print(merged_df)

      image_id  bbox_id                                          keypoints  \
0          139        0  [[429.280517578125, 169.61883544921875, 0.8657...   
1          785        0  [[367.1922302246094, 80.87351989746094, 0.9483...   
2          872        0  [[214.49267578125, 184.88751220703125, 0.79049...   
3          885        2  [[632.353759765625, 46.84330749511719, 0.96756...   
4          885        0  [[373.1753234863281, 217.2002716064453, 0.9457...   
...        ...      ...                                                ...   
6329    580294        0  [[587.7160034179688, 73.6966552734375, 0.31208...   
6330    581062        0  [[163.4222869873047, 66.69705200195312, 0.9830...   
6331    581206        0  [[347.857177734375, 109.86737060546875, 0.1125...   
6332    581317        0  [[501.866455078125, 118.10910034179688, 0.9580...   
6333    581357        0  [[338.90496826171875, 165.29612731933594, 0.96...   

                                        center  \
0     [438.82

In [150]:
merged_df[merged_df['bbox_id']!=0]

,image_id,bbox_id,keypoints,center,scale,area,score
3,885,2,"[[632.353759765625, 46.84330749511719, 0.96756...","[616.864990234375, 139.02000427246094]","[1.0616250038146973, 1.4155000448226929]",60109.207031,0.805771
5,885,1,"[[378.148681640625, 110.68104553222656, 0.9212...","[336.43499755859375, 174.0850067138672]","[0.7900781035423279, 1.0534374713897705]",33291.917969,0.886572
6,1000,8,"[[452.5885009765625, 138.02200317382812, 0.920...","[454.2950134277344, 167.11000061035156]","[0.5632500052452087, 0.7509999871253967]",16920.029297,0.661670
7,1000,1,"[[423.144775390625, 151.42755126953125, 0.7558...","[423.9949951171875, 142.67999267578125]","[0.22581249475479126, 0.3010833263397217]",2719.535156,0.689166
8,1000,11,"[[374.941162109375, 147.42462158203125, 0.9146...","[375.30999755859375, 138.77499389648438]","[0.3227499723434448, 0.43033331632614136]",5555.603027,0.735997
...,...,...,...,...,...,...,...
6318,579070,4,"[[43.96455383300781, -0.25601959228515625, 0.5...","[52.93000030517578, 44.30500030517578]","[0.6485000252723694, 0.8646667003631592]",22429.455078,0.587185
6319,579070,3,"[[543.1754150390625, 208.23660278320312, 0.996...","[566.4099731445312, 223.6649932861328]","[1.0175156593322754, 1.3566875457763672]",55218.035156,0.648577
6321,579070,2,"[[181.37167358398438, 107.21159362792969, 0.97...","[196.38999938964844, 130.02999877929688]","[1.1873749494552612, 1.5831667184829712]",75192.500000,0.818256
6322,579307,1,"[[303.63323974609375, 281.7812805175781, 0.967...","[317.3450012207031, 430.9599914550781]","[1.950374960899353, 2.6005001068115234]",202878.015625,0.835583


In [151]:
merged_df.center

0       [438.82501220703125, 226.11500549316406]
1        [389.6400146484375, 217.57000732421875]
2         [290.7349853515625, 328.8450012207031]
3         [616.864990234375, 139.02000427246094]
4         [346.8550109863281, 293.6000061035156]
                          ...                   
6329     [525.0700073242188, 220.07000732421875]
6330     [162.58999633789062, 107.5250015258789]
6331    [234.26499938964844, 109.66000366210938]
6332    [511.06500244140625, 204.46499633789062]
6333    [372.20001220703125, 216.56500244140625]
Name: center, Length: 6334, dtype: object

In [152]:
gt_df = gt_df.sort_values(by='image_id', ascending=True).reset_index(drop=True)
gt_df

,image_id,bbox,width,height,bbox_size,x,y,center,scale
0,139,"[384.43, 172.21, 15.12, 35.74]",15.12,35.74,540.3888,384.43,172.21,"[393.34488, 193.39546]","[0.223375, 0.223375]"
1,139,"[412.8, 157.61, 53.05, 138.01]",53.05,138.01,7321.4305,412.80,157.61,"[439.325, 226.615]","[0.86256254, 0.86256254]"
2,785,"[280.79, 44.73, 218.7, 346.68]",218.70,346.68,75818.9160,280.79,44.73,"[382.92648, 233.4096]","[2.16675, 2.16675]"
3,872,"[145.26, 100.67, 291.95, 457.35]",291.95,457.35,133523.3325,145.26,100.67,"[291.235, 329.345]","[2.8584375, 2.8584375]"
4,872,"[163.73, 126.42, 265.69, 480.4]",265.69,480.40,127637.4760,163.73,126.42,"[247.06969, 424.5918]","[3.0025, 3.0025]"
...,...,...,...,...,...,...,...,...,...
10999,581357,"[19.53, 261.51, 29.31, 76.62]",29.31,76.62,2245.7322,19.53,261.51,"[34.185, 299.82]","[0.478875, 0.478875]"
11000,581357,"[180.89, 440.51, 21.1, 38.36]",21.10,38.36,809.3960,180.89,440.51,"[191.44, 459.69]","[0.23975, 0.23975]"
11001,581357,"[230.28, 420.86, 19.02, 37.35]",19.02,37.35,710.3970,230.28,420.86,"[239.79, 439.535]","[0.2334375, 0.2334375]"
11002,581357,"[487.56, 412.17, 20.12, 23.31]",20.12,23.31,468.9972,487.56,412.17,"[497.62, 423.825]","[0.14568749, 0.14568749]"


In [153]:
merged_df.head()

,image_id,bbox_id,keypoints,center,scale,area,score
0,139,0,"[[429.280517578125, 169.61883544921875, 0.8657...","[438.82501220703125, 226.11500549316406]","[0.6422343850135803, 0.8563125133514404]",21998.134766,0.763357
1,785,0,"[[367.1922302246094, 80.87351989746094, 0.9483...","[389.6400146484375, 217.57000732421875]","[1.6203750371932983, 2.1605000495910645]",140032.812500,0.902015
2,872,0,"[[214.49267578125, 184.88751220703125, 0.79049...","[290.7349853515625, 328.8450012207031]","[2.1391406059265137, 2.852187395095825]",244049.203125,0.664442
3,885,2,"[[632.353759765625, 46.84330749511719, 0.96756...","[616.864990234375, 139.02000427246094]","[1.0616250038146973, 1.4155000448226929]",60109.207031,0.805771
4,885,0,"[[373.1753234863281, 217.2002716064453, 0.9457...","[346.8550109863281, 293.6000061035156]","[0.9713437557220459, 1.2951250076293945]",50320.460938,0.843057


In [154]:
gt_df.head()

,image_id,bbox,width,height,bbox_size,x,y,center,scale
0,139,"[384.43, 172.21, 15.12, 35.74]",15.12,35.74,540.3888,384.43,172.21,"[393.34488, 193.39546]","[0.223375, 0.223375]"
1,139,"[412.8, 157.61, 53.05, 138.01]",53.05,138.01,7321.4305,412.80,157.61,"[439.325, 226.615]","[0.86256254, 0.86256254]"
2,785,"[280.79, 44.73, 218.7, 346.68]",218.70,346.68,75818.9160,280.79,44.73,"[382.92648, 233.4096]","[2.16675, 2.16675]"
3,872,"[145.26, 100.67, 291.95, 457.35]",291.95,457.35,133523.3325,145.26,100.67,"[291.235, 329.345]","[2.8584375, 2.8584375]"
4,872,"[163.73, 126.42, 265.69, 480.4]",265.69,480.40,127637.4760,163.73,126.42,"[247.06969, 424.5918]","[3.0025, 3.0025]"


In [155]:
from scipy.spatial.distance import cdist

def find_closest_gt(row, gt_df):
    if not pd.isna(row['center']).any():
        center = row['center']
        gt_for_image = gt_df[gt_df['image_id'] == row['image_id']]
        distances = cdist([center], gt_for_image['center'].tolist())
        closest_index = distances.argmin()
        closest_gt_row = gt_for_image.iloc[closest_index]
        return closest_gt_row
    else:
        return pd.Series([pd.NA] * len(gt_df.columns))

# merged_df의 각 행에 대해 가장 가까운 gt_df의 값을 찾아서 병합
df = pd.concat([merged_df, merged_df.apply(find_closest_gt, gt_df=gt_df, axis=1)], axis=1)

In [156]:
df.head()

,image_id,bbox_id,keypoints,center,scale,area,score,image_id,bbox,width,height,bbox_size,x,y,center,scale
0,139,0,"[[429.280517578125, 169.61883544921875, 0.8657...","[438.82501220703125, 226.11500549316406]","[0.6422343850135803, 0.8563125133514404]",21998.134766,0.763357,139,"[412.8, 157.61, 53.05, 138.01]",53.05,138.01,7321.4305,412.80,157.61,"[439.325, 226.615]","[0.86256254, 0.86256254]"
1,785,0,"[[367.1922302246094, 80.87351989746094, 0.9483...","[389.6400146484375, 217.57000732421875]","[1.6203750371932983, 2.1605000495910645]",140032.812500,0.902015,785,"[280.79, 44.73, 218.7, 346.68]",218.70,346.68,75818.9160,280.79,44.73,"[382.92648, 233.4096]","[2.16675, 2.16675]"
2,872,0,"[[214.49267578125, 184.88751220703125, 0.79049...","[290.7349853515625, 328.8450012207031]","[2.1391406059265137, 2.852187395095825]",244049.203125,0.664442,872,"[145.26, 100.67, 291.95, 457.35]",291.95,457.35,133523.3325,145.26,100.67,"[291.235, 329.345]","[2.8584375, 2.8584375]"
3,885,2,"[[632.353759765625, 46.84330749511719, 0.96756...","[616.864990234375, 139.02000427246094]","[1.0616250038146973, 1.4155000448226929]",60109.207031,0.805771,885,"[595.68, 25.78, 43.37, 227.48]",43.37,227.48,9865.8076,595.68,25.78,"[617.365, 139.52]","[1.4217501, 1.4217501]"
4,885,0,"[[373.1753234863281, 217.2002716064453, 0.9457...","[346.8550109863281, 293.6000061035156]","[0.9713437557220459, 1.2951250076293945]",50320.460938,0.843057,885,"[277.31, 189.99, 140.09, 208.22]",140.09,208.22,29169.5398,277.31,189.99,"[347.355, 294.1]","[1.301375, 1.301375]"


In [157]:
df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'area', 'score',
       'image_id', 'bbox', 'width', 'height', 'bbox_size', 'x', 'y', 'center',
       'scale'],
      dtype='object')

In [158]:
# 중복된 열 이름 확인
duplicated_columns = df.columns[df.columns.duplicated()].tolist()

# 이름이 같은 열 중 첫 번째 열만 남기고 나머지는 제거
filtered_df = df.loc[:, ~df.columns.duplicated()]

In [159]:
filtered_df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'area', 'score',
       'bbox', 'width', 'height', 'bbox_size', 'x', 'y'],
      dtype='object')

In [160]:
filtered_df.to_csv("merged_df.csv", index=False)